In [17]:
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
data = pd.read_csv('gladilin/train_set.csv', encoding="utf8").sort_values('transaction_date', ascending=True)

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
987707,4.163903,SAVUSHKINA 141 TK MERKURY UNIVERSAM PATERS\ST....,NaN,NaN,ST.PETERSBURG,RUS,643.0,e40a8e2201ef64e9f9c833dcd7fcda61,59.998,30.196,6011,NaN,NaN,NaN,4fa774768d11b5db2259e3beff2a4b2b,2017-01-27,60.000,30.209
1139228,3.706154,RUS,NaN,NaN,MOSCOW,RUS,643.0,95e0109e67771baa2e5cbeb1df7a394f,55.778,37.616,6011,NaN,NaN,NaN,64df0fecdb24aa9bb165b667267becce,2017-01-27,55.778,37.623
1016179,2.609930,"KVA, NOVOJASENEVSKIJ PR-KT, D. 1, TRC SPEKTR 2...",NaN,NaN,MOSCOW,RUS,643.0,999f29548d67933a85c9486cc7fdb220,55.681,37.664,6011,NaN,NaN,NaN,8706107cf971664e81e87b2d50c54756,2017-01-27,NaN,NaN
945894,4.098726,"25 Oktyabrya, d. 9\Gatchina\188300 RUSRUS",NaN,NaN,Gatchina,RUS,643.0,53814b7cd2bc49de90cbf133e330e111,59.367,30.111,6011,NaN,NaN,NaN,036753e7b6e45dc70636ba65ba3ba9a1,2017-01-27,NaN,NaN
1067538,3.290021,"ALAHNA, UL. LESOPILNAJA, 1, TC GALAKTIKA\NIZHN...",NaN,NaN,NIZHNIY NOVGO,RUS,643.0,babe535fc094253e3eb2774969bccd6f,56.471,43.550,6011,NaN,NaN,NaN,e84cde01a9012acac932af4deb7596cb,2017-01-27,NaN,NaN


<ul>
    <li>В каких регионах больше относительное число людей с вредными привычками (категории рестораны-алкогольная продукция, табак и т.д.)?..</li>
<li>Сколько будет совершено транзакций в следующем году в категории товаров “Рестораны и кафе” в Московском регионе?..</li>
<li>Как соотносятся множества людей, часто посещающих рестораны-кафе с людьми, часто посещающих аптеки?..</li>
    <li>Как часто люди ходят в рестораны/кафе рядом с работой? рядом с домом?..</li>
<li>Есть ли корреляция между количеством транзакций в категории алкоголь и  табак и и количеством покупок в аптеках?..</li>
<li>Влияют ли занятия спортом (==транзакции в категории “Спорт”) на вредные привычке в различных регионах?..</li>
<li>У кого более выражены вредные привычки: у тратящих много и часто или наоборот?..</li>
</ul>

In [5]:
# MCC
APTEC = 5912 # Аптеки
CAFE = 5812 #Общепит и рестораны
BAR = 5813 #Бары

In [6]:
A = data[data['mcc'] == APTEC]
B = data[data['mcc'] == BAR]
C = data[data['mcc'] == CAFE]

In [19]:
group_param = 'customer_id'

In [20]:
def plot_group(series):
    g=series.groupby(group_param)
    x=[]
    y=[]
    mean = pd.Series([s.size for n,s in g]).mean()
    for name, group in g:
        if group.size > mean:
            x.append(name)
            y.append(group.size)
    
    plt.plot(x,y)

In [21]:
def more_then_mean(series):
    g=series.groupby(group_param)
    names=[]
    y=[]
    mean = pd.Series([s.size for n,s in g]).mean()
    for name, group in g:
        if group.size > mean:
            names.append(name)
    
    return names

In [25]:
def compare(A, B):
    a = more_then_mean(A)
    b = more_then_mean(B)
    clients = []

    for client in a:
        if client not in b:
            pass
        else:
            clients.append(client)
    return {'x': len(a), 'y': len(b), 'z': len(clients)}

In [50]:
import numpy as np
results = []
unique = np.concatenate([data['mcc'].unique()[:10], np.array([APTEC, CAFE])])
for i, mcc in enumerate(unique):
    A = data[data['mcc']==mcc]
    for j, m in enumerate(unique):
        if j < i:
            continue
        B = data[data['mcc']==m]
        if mcc != m and [int(mcc),int(m)] not in [d['pair'][::-1] for d in results]:
            result = compare(A, B)
            result['pair'] = [int(mcc), int(m)]
            results.append(result)
        
print(len(results), results[0])

60 {'x': 3654, 'y': 22, 'z': 9, 'pair': [6011, 6010]}


In [52]:
print(len(results))
d = compare(data[data['mcc']==APTEC], data[data['mcc']==CAFE])
d['pair'] = [APTEC, CAFE]
print(d)
results.append(d)
print(len(results))

60
{'x': 2448, 'y': 1336, 'z': 722, 'pair': [5912, 5812]}
61


In [53]:
import json
with open('data/data.json', 'w') as outfile:
    json.dump(results, outfile)

In [47]:
results = []
unique = data['mcc'].unique()
for mcc in unique:
    A = data[data['mcc']==mcc]
    for m in unique:
        B = data[data['mcc']==m]
        if mcc != m:
            result = compare(A, B)
            result['pair'] = [name, n]
            results.append(result)
        
print(len(results), results[0])

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(20,10))

print('aptec')
plot_group(A)
#print('bar')
#plot_group(B)
print('cafe')
plot_group(C)

#plt.legend(labels=['Aptecs', 'Bars', 'Cafes'])

aptec
bar
cofe
